### Load libraries

In [136]:
import polars as pl
import lightgbm as lgm
import numpy as np
import pandas as pd

SEMILLA = 800089


In [137]:
# Load a CSV and create a DataFrame
df = pl.read_csv('~/buckets/b1/datasets/dataset_pequeno.csv', ignore_errors=True)


In [138]:
df.head(5)


numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,tcuentas,ccuenta_corriente,mcuenta_corriente_adicional,mcuenta_corriente,ccaja_ahorro,mcaja_ahorro,mcaja_ahorro_adicional,mcaja_ahorro_dolares,cdescubierto_preacordado,mcuentas_saldo,ctarjeta_debito,ctarjeta_debito_transacciones,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,ctarjeta_master,ctarjeta_master_transacciones,mtarjeta_master_consumo,cprestamos_personales,mprestamos_personales,cprestamos_prendarios,mprestamos_prendarios,cprestamos_hipotecarios,…,Master_mconsumospesos,Master_mconsumosdolares,Master_mlimitecompra,Master_madelantopesos,Master_madelantodolares,Master_fultimo_cierre,Master_mpagado,Master_mpagospesos,Master_mpagosdolares,Master_fechaalta,Master_mconsumototal,Master_cconsumos,Master_cadelantosefectivo,Master_mpagominimo,Visa_delinquency,Visa_status,Visa_mfinanciacion_limite,Visa_Fvencimiento,Visa_Finiciomora,Visa_msaldototal,Visa_msaldopesos,Visa_msaldodolares,Visa_mconsumospesos,Visa_mconsumosdolares,Visa_mlimitecompra,Visa_madelantopesos,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,i64,f64,f64,f64,i64,f64,i64,i64,f64,i64,i64,f64,i64,i64,f64,i64,f64,i64,f64,i64,…,f64,f64,f64,i64,i64,i64,f64,f64,f64,i64,f64,i64,i64,f64,i64,i64,f64,i64,str,f64,f64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,i64,f64,i64,i64,f64,str
31116053,202107,1,0,0,50,201,13073.36,161035.99,3163.52,5554.23,2894.06,8,1,1,0,-1324.9,2,54545.59,0.0,1.0715e6,1,1.2235e6,1,15,15370.99,1,126,161078.91,1,54,92208.79,0,0.0,0,0.0,0,…,73018.61,0.0,1.5550e6,0,0,10,0.0,-22873.5,0.0,6060,73018.61,34,0,28386.6,0,0,1399475.5,-517,null,596694.3,695387.54,4534.86,121585.39,34.48,1.5550e6,0,0,10,0.0,-82110.0,0.0,6060,121585.39,75,0,77617.41,"""CONTINUA"""
31116803,202107,1,0,0,59,326,2640.48,56751.51,5232.22,-4483.75,2274.25,9,1,1,0,20105.36,2,76350.74,0.0,0.0,1,99393.19,2,0,0.0,1,8,49943.72,1,9,75147.77,0,0.0,0,0.0,0,…,62071.17,0.0,394128.0,0,0,24,75147.77,-74768.49,0.0,4219,62071.17,7,0,3284.4,0,0,506736.0,-913,null,43646.39,49496.83,1700.38,21596.4,15.15,563040.0,0,0,3,0.0,-85347.94,17.59,4211,21596.4,7,0,4938.33,"""CONTINUA"""
31117730,202107,1,0,0,68,272,1343.51,20201.31,582.94,-780.79,1429.19,8,1,1,0,0.0,2,104101.97,0.0,0.0,1,104102.23,2,0,0.0,1,3,4481.25,0,0,0.0,0,0.0,0,0.0,0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0,0,63342.0,-1917,null,3712.47,4354.72,0.0,1529.02,0.0,70380.0,0,0,24,4692.0,-4692.0,0.0,7951,1529.02,2,0,1407.6,"""CONTINUA"""
31117908,202107,1,0,0,80,326,5523.19,55827.28,2712.26,-498.62,2894.98,9,1,1,0,0.0,2,203511.58,0.0,4.5381e6,1,4.7940e6,3,0,0.0,1,17,21110.7,1,3,8848.31,0,0.0,0,0.0,0,…,0.0,0.0,1.5482e6,0,0,24,8857.15,-8857.15,0.0,5773,0.0,0,0,351.9,0,0,1.4028e6,-1095,null,18015.19,21131.81,0.0,12815.99,0.0,1558682.4,0,0,3,0.0,-26420.0,0.0,7702,12815.99,8,0,856.29,"""CONTINUA"""
31117977,202107,1,0,0,60,261,4819.35,37386.4,1878.4,-4232.87,6740.07,8,1,1,0,0.0,2,342041.99,0.0,68286.2,1,397138.73,3,66,149763.44,1,26,172251.2,1,0,0.0,0,0.0,0,0.0,0,…,null,null,849071.68,null,null,10,0.0,null,null,7368,null,null,null,0.0,0,0,1.0689e6,-456,null,148391.92,172264.43,1799.28,157186.12,7.7,1.1877e6,0,0,24,174063.72,-160642.13,12.89,7949,157186.12,27,0,9548.22,"""CONTINUA"""


In [139]:
df.shape

(329919, 155)

Seteo los meses que se van a usar como entrenamiento y cuales para evaluar la ganancia

In [140]:
training_month = [ 202107 ]
testing_month = [ 202109 ]

In [141]:
set(df['clase_ternaria'])

{'BAJA+1', 'BAJA+2', 'CONTINUA', None}

In [142]:
df = df.with_columns(pl.col("clase_ternaria")\
    .map_elements(lambda x : 0 if x == "CONTINUA" else 1, None)\
    .alias("clase01"))

In [143]:
set(df['clase01'])

{0, 1, None}

In [144]:
df.head(5)

numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,tcuentas,ccuenta_corriente,mcuenta_corriente_adicional,mcuenta_corriente,ccaja_ahorro,mcaja_ahorro,mcaja_ahorro_adicional,mcaja_ahorro_dolares,cdescubierto_preacordado,mcuentas_saldo,ctarjeta_debito,ctarjeta_debito_transacciones,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,ctarjeta_master,ctarjeta_master_transacciones,mtarjeta_master_consumo,cprestamos_personales,mprestamos_personales,cprestamos_prendarios,mprestamos_prendarios,cprestamos_hipotecarios,…,Master_mconsumosdolares,Master_mlimitecompra,Master_madelantopesos,Master_madelantodolares,Master_fultimo_cierre,Master_mpagado,Master_mpagospesos,Master_mpagosdolares,Master_fechaalta,Master_mconsumototal,Master_cconsumos,Master_cadelantosefectivo,Master_mpagominimo,Visa_delinquency,Visa_status,Visa_mfinanciacion_limite,Visa_Fvencimiento,Visa_Finiciomora,Visa_msaldototal,Visa_msaldopesos,Visa_msaldodolares,Visa_mconsumospesos,Visa_mconsumosdolares,Visa_mlimitecompra,Visa_madelantopesos,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria,clase01
i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,i64,f64,f64,f64,i64,f64,i64,i64,f64,i64,i64,f64,i64,i64,f64,i64,f64,i64,f64,i64,…,f64,f64,i64,i64,i64,f64,f64,f64,i64,f64,i64,i64,f64,i64,i64,f64,i64,str,f64,f64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,i64,f64,i64,i64,f64,str,i64
31116053,202107,1,0,0,50,201,13073.36,161035.99,3163.52,5554.23,2894.06,8,1,1,0,-1324.9,2,54545.59,0.0,1.0715e6,1,1.2235e6,1,15,15370.99,1,126,161078.91,1,54,92208.79,0,0.0,0,0.0,0,…,0.0,1.5550e6,0,0,10,0.0,-22873.5,0.0,6060,73018.61,34,0,28386.6,0,0,1399475.5,-517,null,596694.3,695387.54,4534.86,121585.39,34.48,1.5550e6,0,0,10,0.0,-82110.0,0.0,6060,121585.39,75,0,77617.41,"""CONTINUA""",0
31116803,202107,1,0,0,59,326,2640.48,56751.51,5232.22,-4483.75,2274.25,9,1,1,0,20105.36,2,76350.74,0.0,0.0,1,99393.19,2,0,0.0,1,8,49943.72,1,9,75147.77,0,0.0,0,0.0,0,…,0.0,394128.0,0,0,24,75147.77,-74768.49,0.0,4219,62071.17,7,0,3284.4,0,0,506736.0,-913,null,43646.39,49496.83,1700.38,21596.4,15.15,563040.0,0,0,3,0.0,-85347.94,17.59,4211,21596.4,7,0,4938.33,"""CONTINUA""",0
31117730,202107,1,0,0,68,272,1343.51,20201.31,582.94,-780.79,1429.19,8,1,1,0,0.0,2,104101.97,0.0,0.0,1,104102.23,2,0,0.0,1,3,4481.25,0,0,0.0,0,0.0,0,0.0,0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,0,0,63342.0,-1917,null,3712.47,4354.72,0.0,1529.02,0.0,70380.0,0,0,24,4692.0,-4692.0,0.0,7951,1529.02,2,0,1407.6,"""CONTINUA""",0
31117908,202107,1,0,0,80,326,5523.19,55827.28,2712.26,-498.62,2894.98,9,1,1,0,0.0,2,203511.58,0.0,4.5381e6,1,4.7940e6,3,0,0.0,1,17,21110.7,1,3,8848.31,0,0.0,0,0.0,0,…,0.0,1.5482e6,0,0,24,8857.15,-8857.15,0.0,5773,0.0,0,0,351.9,0,0,1.4028e6,-1095,null,18015.19,21131.81,0.0,12815.99,0.0,1558682.4,0,0,3,0.0,-26420.0,0.0,7702,12815.99,8,0,856.29,"""CONTINUA""",0
31117977,202107,1,0,0,60,261,4819.35,37386.4,1878.4,-4232.87,6740.07,8,1,1,0,0.0,2,342041.99,0.0,68286.2,1,397138.73,3,66,149763.44,1,26,172251.2,1,0,0.0,0,0.0,0,0.0,0,…,null,849071.68,null,null,10,0.0,null,null,7368,null,null,null,0.0,0,0,1.0689e6,-456,null,148391.92,172264.43,1799.28,157186.12,7.7,1.1877e6,0,0,24,174063.72,-160642.13,12.89,7949,157186.12,27,0,9548.22,"""CONTINUA""",0


Transformo las columnas en formato `str` a número


In [145]:
df = df.with_columns(pl.col('Master_Finiciomora')\
    .map_elements(lambda x : int(x) if x != "None" else None)\
    .alias("Master_Finiciomora"))
df = df.with_columns(pl.col('Visa_Finiciomora')\
    .map_elements(lambda x : int(x) if x != "None" else None)\
    .alias("Visa_Finiciomora"))

Filtro las filas del row en datos de training y de testing

In [146]:
training_df = df.filter(pl.col("foto_mes").is_in(training_month))
testing_df = df.filter(pl.col("foto_mes").is_in(testing_month))

In [147]:
print(f'{ training_df.shape = }')
print(f'{ testing_df.shape = }')

 training_df.shape = (164682, 156)
 testing_df.shape = (165237, 156)


In [148]:
Y = training_df['clase01'].to_pandas()
ignored_indep_feat = [ "foto_mes", "clase_ternaria", "clase01" ]
X = training_df.drop(ignored_indep_feat).to_pandas()


In [149]:
X_eval = testing_df.drop(ignored_indep_feat).to_pandas()

In [150]:
print(f'{ X.shape = }')
print(f'{ Y.shape = }')
print(f'{ X_eval.shape = }')



 X.shape = (164682, 153)
 Y.shape = (164682,)
 X_eval.shape = (165237, 153)


In [162]:
# Execute lightgbm with the default hyperparameters
model = lgm.LGBMRegressor(
    objective = "binary",
    max_bin=31,
    learning_rate=0.0100746999,
    feature_fraction=0.5144127527,
    num_iterations=559,
    min_data_in_leaf=505,
    num_leaves=44,
    seed = SEMILLA
)


In [163]:
# Create train and test splits
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [164]:


model.fit(X, Y, eval_set=[(X_test, Y_test)], eval_metric='l1')

/Users/fede/.local/share/virtualenvs/src_py-DJPQCuwV/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.5144127527, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5144127527
[LightGBM] [Warning] min_data_in_leaf is set=505, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=505
[LightGBM] [Warning] feature_fraction is set=0.5144127527, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5144127527
[LightGBM] [Warning] min_data_in_leaf is set=505, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=505
[LightGBM] [Info] Number of positive: 2346, number of negative: 162336
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027453 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3364
[LightGBM] [Info] Number of data points in the train set: 164682, number of used features: 142
[LightGBM] [Warning] feature_fraction is s

LGBMRegressor(feature_fraction=0.5144127527, learning_rate=0.0100746999,
              max_bin=31, min_data_in_leaf=505, num_iterations=559,
              num_leaves=44, objective='binary', seed=800089)

In [165]:
Y_eval = model.predict(X_eval)

[LightGBM] [Warning] feature_fraction is set=0.5144127527, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5144127527
[LightGBM] [Warning] min_data_in_leaf is set=505, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=505


In [169]:
prediction = pl.DataFrame({
    'numero_de_cliente': X_eval['numero_de_cliente'],
    'Prob': Y_eval
})
prediction = prediction.sort(by="Prob", descending=True)


In [170]:
prediction.head(10)

numero_de_cliente,Prob
i64,f64
185782164,0.824916
92289921,0.810047
145816140,0.801692
61607602,0.770551
138311471,0.751978
87299159,0.748458
150317301,0.747096
146577767,0.725225
192181921,0.705728


In [175]:
csv_src = prediction['numero_de_cliente'].to_frame().with_row_count("row_nr")



In [181]:
for i in range(8000, 12000, 500):
    csv_src.with_columns(pl.col("row_nr")\
        .map_elements(lambda x : 1 if x < i else 0)\
        .alias("Predicted"))\
        .drop("row_nr")\
        .write_csv(f'prediction_{i}.csv')